In [1]:
import pandas as pd
import numpy as np
import sys
import os 
#from ...util.model.training import get_percent

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
%pwd

'c:\\Users\\lowmi\\OneDrive\\Desktop\\Uni\\DSA4262\\DSA4262-ACMXZ\\preprocessing\\notebooks'

In [4]:
sys.path.append(os.path.abspath('../../util/model'))
from training import get_percent

In [5]:
%pwd

'c:\\Users\\lowmi\\OneDrive\\Desktop\\Uni\\DSA4262\\DSA4262-ACMXZ\\preprocessing\\notebooks'

In [6]:
info_data_path = "../../data/raw_data/data.info"
with open(info_data_path, 'r') as f:
    info = f.read().splitlines()
data_df1 = pd.read_parquet("../../data/merged_data/merged_data_v2_1.parquet")
data_df2 = pd.read_parquet("../../data/merged_data/merged_data_v2_2.parquet")
data_df3 = pd.read_parquet("../../data/merged_data/merged_data_v2_3.parquet")


In [7]:
print(data_df1.shape)
print(data_df2.shape)
print(data_df3.shape)


(5000000, 15)
(5000000, 15)
(1027106, 15)


In [8]:
raw_df =  pd.concat([data_df1, data_df2, data_df3])

In [9]:
raw_df.head()

,transcript,position,nucleotides,reads_count,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1,gene_id,label
0,ENST00000000233,244,AAGACCA,185,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,185,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,185,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,185,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,185,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0


In [10]:
def model_features_and_clean(df):
    '''Function to select features for modelling and clean the initial raw data. 
        Removes rows which contain NAN values in key_columns, and converts all NAN values to 0 for non_nan_cols

    :Parameters:
    ------------
        self.df: DataFrame
            DataFrame containing raw data

    :Returns:
    ---------
        self.df: DataFrame
            DataFrame containing features required for modelling
    '''
    model_features_list =  ['transcript', 'position', 'nucleotides', 'reads_count', 'gene_id',
                            'dwellingtime_-1', 'std_-1', 'mean_-1',
                            'dwellingtime_0', 'std_0', 'mean_0',
                            'dwellingtime_+1', 'std_+1', 'mean_+1','label']
    non_nan_cols =  []
    key_columns =['label']
    df = df[model_features_list]
    df[non_nan_cols] = df[non_nan_cols].fillna(0)
    df = df.dropna(subset=key_columns)
    return df

In [11]:
cleaned_df = model_features_and_clean(raw_df)

In [12]:
cleaned_df.head()

,transcript,position,nucleotides,reads_count,gene_id,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1,label
0,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,0
1,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,0
2,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,0
3,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,0
4,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,0


In [13]:
# function to clean
def feature_eng(df):
    temp = pd.DataFrame(df.groupby(['gene_id', 'transcript', 'position', 'nucleotides', 'reads_count', 'label'], as_index=False)
                           .agg({'dwellingtime_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'dwellingtime_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'dwellingtime_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean]}))
    temp.columns = ['gene_id', 'transcript', 'position', 'nucleotides', 'reads_count', 'label',
                        'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                        'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean',
                        'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean',
                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                        'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean',
                        'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean',
                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                        'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean',
                        'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']
    return temp

In [14]:
raw_df.columns

Index(['transcript', 'position', 'nucleotides', 'reads_count',
       'dwellingtime_-1', 'std_-1', 'mean_-1', 'dwellingtime_0', 'std_0',
       'mean_0', 'dwellingtime_+1', 'std_+1', 'mean_+1', 'gene_id', 'label'],
      dtype='object')

In [15]:
percentiles_df = feature_eng(cleaned_df)

In [16]:
percentiles_df.head()

,gene_id,transcript,position,nucleotides,reads_count,label,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,ENSG00000000003,ENST00000373020,1006,TAGACCT,21,0,0.007650,0.009760,0.015100,0.010726,2.8900,3.100,3.3400,3.021905,118.000,119.00,120.00,118.952381,0.005980,0.009090,0.014900,0.011677,4.3000,5.600,7.5800,6.000000,125.00,127.00,128.00,127.000000,0.003650,0.005980,0.008300,0.006490,2.0600,3.210,5.1700,3.900476,75.800,77.10,77.800,76.857143
1,ENSG00000000003,ENST00000373020,1013,AAAACTA,20,0,0.007965,0.009800,0.013175,0.011538,2.1925,2.460,2.7325,2.471500,108.000,109.00,110.00,108.900000,0.003320,0.005305,0.007763,0.006050,3.8850,4.415,4.8200,4.091500,100.00,102.00,105.00,102.675000,0.006112,0.008840,0.011600,0.010213,2.4775,2.870,4.0100,3.299000,92.125,93.25,94.975,93.765000
2,ENSG00000000003,ENST00000373020,1149,GAAACAC,22,0,0.004917,0.007155,0.009473,0.008890,2.2100,3.030,4.2975,3.440909,104.000,105.00,107.00,105.363636,0.004958,0.006885,0.010733,0.008051,1.9875,2.570,2.9875,2.652227,96.80,98.50,100.75,99.081818,0.003928,0.006205,0.007832,0.006491,1.6300,1.855,2.2675,2.028182,89.450,90.65,92.600,90.777273
3,ENSG00000000003,ENST00000373020,512,ATAACTC,20,0,0.004040,0.005990,0.008387,0.007247,1.7400,2.325,2.7050,2.359500,85.775,87.05,87.35,86.795000,0.003980,0.007735,0.013375,0.011177,2.0800,2.375,2.7300,2.496500,97.05,98.05,99.05,97.965000,0.004870,0.006870,0.011200,0.008500,1.3900,1.570,1.8750,1.588750,92.100,92.55,93.325,92.330000
4,ENSG00000000003,ENST00000373020,689,TAAACAA,21,0,0.005310,0.007640,0.014300,0.009868,1.7500,1.980,2.2300,1.972857,102.000,103.00,104.00,102.495238,0.003650,0.006420,0.008960,0.007332,1.9600,2.430,2.9100,2.551905,96.20,98.70,99.70,97.928571,0.004580,0.005730,0.008590,0.006917,1.7400,2.190,2.3700,2.198571,88.500,89.60,90.600,89.385714


In [59]:
# raw_df.loc[(raw_df['nucleotides']=='AAGACTG') & (raw_df['position'] == '2996') & (raw_df['transcript'] == 'ENST00000621914') ]

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
def split_stratified_into_train_val_test(df, random_state=None):

    # getting unique id_col and stratify_col for splitting
    id_col = ['gene_id']
    stratify_col = 'label'
    train_percent = 0.8 # percentage of train data
    validation_percent = 0.1 # percentage of validation data
    test_percent = 0.1 # percentage of test data
    #position_col = ['transcript', 'position']
    target_col = 'label'

    temp_col = id_col + [stratify_col]
    df_target = df[temp_col].drop_duplicates()
    X = df_target  
    y = df_target[[stratify_col]]
    

    # parameter checks
    if train_percent + validation_percent + test_percent != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' %
                            (train_percent, validation_percent, test_percent))

    if stratify_col not in df_target.columns:
        raise ValueError('%s is not a column in the dataframe' %
                            (stratify_col))

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X, y, stratify=y,
                                                                    test_size=(
                                                                        1.0 - train_percent),
                                                                    random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_test_percent = test_percent / \
        (validation_percent + test_percent)
    df_val, df_test, y_val, y_test = train_test_split(df_temp, y_temp,
                                                        stratify=y_temp,
                                                        test_size=relative_test_percent,
                                                        random_state=random_state)

    assert len(df_target) == len(df_train) + \
        len(df_val) + len(df_test)


    list_train = df_train[id_col]
    y_train_n = y_train
    new_train_df = pd.concat([list_train,y_train_n],axis=1)
    a = new_train_df.merge(df,how = "left", left_on = ["gene_id", "label"], 
                    right_on = ["gene_id", "label"])
    list_test = df_test[id_col]
    y_test_n = y_test
    new_test_df = pd.concat([list_test,y_test_n],axis=1)
    b = new_test_df.merge(df,how = "left", left_on = ["gene_id", "label"], 
                    right_on = ["gene_id", "label"])
    list_val = df_val[id_col]
    y_val_n = y_val
    new_val_df = pd.concat([list_val,y_val_n], axis=1)
    c = new_val_df.merge(df,how = "left", left_on = ["gene_id", "label"], 
                    right_on = ["gene_id", "label"])
    
    
    # printing percentages
    print("train target percentage:", len(
        a[a[target_col] == '1'])/len(a))
    print("test target percentage:", len(
        b[b[target_col] == '1'])/len(b))
    print("val target percentage:", len(
     
    c[c[target_col] == '1'])/len(c))


    # separating df from target column: features -> X | target -> y
    X_train = a.drop(
        columns = ['gene_id','transcript','position','label']).reset_index(drop=True)
    y_train = pd.DataFrame(
        a[target_col]).reset_index(drop=True)
    X_val = c.drop(
        columns = ['gene_id','transcript','position','label']).reset_index(drop=True)
    y_val = pd.DataFrame(
        c[target_col]).reset_index(drop=True)
    X_test = b.drop(
        columns = ['gene_id','transcript','position','label']).reset_index(drop=True)
    y_test = pd.DataFrame(
        b[target_col]).reset_index(drop=True)
    # printing df shape
    print("train data shape:", X_train.shape)
    print("validation data shape:", X_val.shape)
    print("test data shape:", X_test.shape)

    return(a, b, c, X_train, y_train,
            X_val, y_val, X_test, y_test, new_val_df, list_train)

In [19]:
df_train, df_test, df_val, X_train, y_train, X_val, y_val, X_test, y_test, df_val_id, list_train = split_stratified_into_train_val_test(percentiles_df, random_state=42)

train target percentage: 0.043292122202922244
test target percentage: 0.0506553911205074
val target percentage: 0.052622910318702136
train data shape: (97870, 38)
validation data shape: (12143, 38)
test data shape: (11825, 38)


In [24]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(97870, 38)
(97870, 1)
(11825, 38)
(11825, 1)
(12143, 38)
(12143, 1)


In [25]:
df_train.head()

,gene_id,label,transcript,position,nucleotides,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,ENSG00000158747,0,ENST00000375136,1300,AAGACCA,48,0.004085,0.00605,0.009362,0.007326,3.370,4.125,4.9675,4.432083,122.0,124.0,125.0,123.229167,0.007865,0.010900,0.015900,0.012958,5.1475,6.410,7.715,6.455208,127.0,128.0,129.25,128.166667,0.005557,0.00764,0.010225,0.008463,1.9275,3.545,4.9325,3.763854,82.00,83.4,84.175,83.087500
1,ENSG00000158747,0,ENST00000375136,1313,GAAACAG,50,0.005855,0.00716,0.010675,0.008841,2.430,2.920,3.3675,3.075400,107.0,108.0,108.0,107.920000,0.003395,0.005405,0.006970,0.006933,2.2425,3.215,4.035,3.257580,95.4,101.5,106.00,100.246000,0.004065,0.00602,0.008960,0.007349,2.4550,3.160,4.6300,3.506040,87.45,89.3,90.775,88.902000
2,ENSG00000158747,0,ENST00000375136,1444,AGGACCC,51,0.005560,0.01050,0.019200,0.011989,5.245,6.080,6.6650,5.923529,115.0,117.0,118.0,116.549020,0.007970,0.013300,0.017400,0.013976,4.6450,5.230,5.915,5.457059,121.0,122.0,123.00,121.784314,0.003980,0.00554,0.007200,0.005917,1.7600,2.270,3.4700,2.767451,80.25,82.2,83.250,81.935294
3,ENSG00000158747,0,ENST00000375136,1453,CTGACTC,49,0.004980,0.00631,0.008300,0.007012,3.430,3.850,4.4100,3.978980,107.0,110.0,110.0,108.816327,0.005980,0.009630,0.013100,0.010202,5.2200,6.390,8.100,6.756735,122.0,124.0,126.00,124.204082,0.004030,0.00651,0.008630,0.006681,2.1100,2.660,3.3400,2.870408,89.60,91.3,93.200,91.830612
4,ENSG00000158747,0,ENST00000375136,1581,GGGACAA,35,0.003485,0.00730,0.014600,0.009141,3.110,4.520,6.9700,5.360000,114.5,117.0,120.0,116.885714,0.003980,0.006640,0.010315,0.008617,6.9700,11.400,12.900,10.157714,109.0,113.0,116.00,112.028571,0.003980,0.00558,0.008590,0.006630,2.0400,2.450,2.8000,2.756571,78.30,79.5,82.200,80.108571


In [26]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC

In [27]:
def oversample_undersample(X_train, y_train, sampling=True):
    '''Function to oversample minority and undersample majority.

    :Returns:
    ---------
        self.X_train_oversampled: DataFrame
            DataFrame contiaining oversampled and understampled training data only for first purchase date excluding target variable
    '''
    X_train_oversampled = X_train.copy()
    undersampling_strategy = 0.12
    oversampling_strategy = 0.17
    seed = 42 # random state

    if sampling:
        undersample = RandomUnderSampler(
            sampling_strategy = undersampling_strategy, random_state=seed)
        oversample = SMOTENC(categorical_features=[
                                0], sampling_strategy=oversampling_strategy, random_state=seed)
        X_train_oversampled, y_train = undersample.fit_resample(
            X_train_oversampled, y_train)
        X_train_oversampled, y_train = oversample.fit_resample(
            X_train_oversampled, y_train)

    return X_train_oversampled, y_train

In [28]:
df_train.head()

,gene_id,label,transcript,position,nucleotides,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,ENSG00000158747,0,ENST00000375136,1300,AAGACCA,48,0.004085,0.00605,0.009362,0.007326,3.370,4.125,4.9675,4.432083,122.0,124.0,125.0,123.229167,0.007865,0.010900,0.015900,0.012958,5.1475,6.410,7.715,6.455208,127.0,128.0,129.25,128.166667,0.005557,0.00764,0.010225,0.008463,1.9275,3.545,4.9325,3.763854,82.00,83.4,84.175,83.087500
1,ENSG00000158747,0,ENST00000375136,1313,GAAACAG,50,0.005855,0.00716,0.010675,0.008841,2.430,2.920,3.3675,3.075400,107.0,108.0,108.0,107.920000,0.003395,0.005405,0.006970,0.006933,2.2425,3.215,4.035,3.257580,95.4,101.5,106.00,100.246000,0.004065,0.00602,0.008960,0.007349,2.4550,3.160,4.6300,3.506040,87.45,89.3,90.775,88.902000
2,ENSG00000158747,0,ENST00000375136,1444,AGGACCC,51,0.005560,0.01050,0.019200,0.011989,5.245,6.080,6.6650,5.923529,115.0,117.0,118.0,116.549020,0.007970,0.013300,0.017400,0.013976,4.6450,5.230,5.915,5.457059,121.0,122.0,123.00,121.784314,0.003980,0.00554,0.007200,0.005917,1.7600,2.270,3.4700,2.767451,80.25,82.2,83.250,81.935294
3,ENSG00000158747,0,ENST00000375136,1453,CTGACTC,49,0.004980,0.00631,0.008300,0.007012,3.430,3.850,4.4100,3.978980,107.0,110.0,110.0,108.816327,0.005980,0.009630,0.013100,0.010202,5.2200,6.390,8.100,6.756735,122.0,124.0,126.00,124.204082,0.004030,0.00651,0.008630,0.006681,2.1100,2.660,3.3400,2.870408,89.60,91.3,93.200,91.830612
4,ENSG00000158747,0,ENST00000375136,1581,GGGACAA,35,0.003485,0.00730,0.014600,0.009141,3.110,4.520,6.9700,5.360000,114.5,117.0,120.0,116.885714,0.003980,0.006640,0.010315,0.008617,6.9700,11.400,12.900,10.157714,109.0,113.0,116.00,112.028571,0.003980,0.00558,0.008590,0.006630,2.0400,2.450,2.8000,2.756571,78.30,79.5,82.200,80.108571


In [29]:
X_train_oversampled, y_train = oversample_undersample(X_train, y_train)

In [30]:
X_train_oversampled.head()

,nucleotides,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,CTAACTG,145,0.00378,0.00502,0.006640,0.005691,1.470,1.73,2.210,1.872103,92.3,93.6,94.8,93.308966,0.004670,0.00622,0.009080,0.007362,2.060,2.38,2.74,2.464531,98.2,99.4,100.0,99.268276,0.003870,0.00531,0.00803,0.006814,1.80,2.11,2.620,2.313655,91.40,92.1,93.10,92.361379
1,CAAACCA,37,0.00454,0.00751,0.008630,0.007982,1.810,2.56,3.330,2.657838,104.0,105.0,106.0,104.300000,0.004650,0.00764,0.011600,0.009066,3.720,4.22,4.82,4.228919,99.4,101.0,102.0,100.632432,0.003650,0.00564,0.00764,0.006361,1.56,1.82,2.060,1.875135,82.30,84.0,84.50,83.764865
2,TTGACCT,31,0.00587,0.00915,0.014050,0.010801,2.735,2.98,3.200,3.071935,106.0,108.0,108.5,106.996774,0.005145,0.00719,0.008735,0.007875,4.565,5.92,7.02,6.285161,120.0,121.0,124.0,121.580645,0.003740,0.00501,0.00764,0.005711,2.03,3.13,3.825,3.299355,81.15,82.7,84.80,82.967742
3,GAAACCA,29,0.00464,0.00598,0.007550,0.006496,2.200,2.56,3.840,3.240000,105.0,106.0,107.0,106.137931,0.003650,0.00598,0.007620,0.006448,3.790,4.39,4.87,4.227241,97.4,99.2,101.0,99.396552,0.003490,0.00465,0.00614,0.005017,1.68,2.13,2.500,2.233241,80.90,82.5,83.40,82.641379
4,CTGACAG,43,0.00415,0.00598,0.009295,0.007353,3.180,3.51,4.085,3.798837,103.0,104.0,106.0,104.525581,0.003980,0.00577,0.008760,0.006542,6.705,8.34,10.95,8.524186,117.0,119.0,122.0,119.372093,0.005275,0.00730,0.00969,0.007783,2.11,2.63,3.510,2.855581,81.10,83.6,85.95,83.360465


In [31]:
X_train_oversampled.shape

(41310, 38)

In [32]:
y_train['label'].value_counts()

0    35308
1     6002
Name: label, dtype: int64

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

In [34]:
X_train_oversampled.columns

Index(['nucleotides', 'reads_count', 'dwelling_time_-1_25',
       'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
       'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25',
       'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25',
       'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
       'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25',
       'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25',
       'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
       'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25',
       'mean_+1_50', 'mean_+1_75', 'mean_+1_mean'],
      dtype='object')

In [35]:
def encoding_train(X_train_oversampled, y_train):


    one_hot_col = ['nucleotides']
    # numeric_cols = ['dwelling_time_25', 'dwelling_time_50', 'dwelling_time_75', 'dwelling_time_mean', 
    #                         'std_25', 'std_50', 'std_75', 'std_mean', 
    #                         'mean_25', 'mean_50', 'mean_75', 'mean_mean']
    numeric_cols = ['reads_count','dwelling_time_-1_25',
                    'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                    'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25',
                    'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25',
                    'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                    'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25',
                    'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25',
                    'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                    'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25',
                    'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']                            

    # piping the encoding
    numeric_encoder = Pipeline([('scale', MinMaxScaler())])
    preprocessor = ColumnTransformer(transformers=[
        ("num", numeric_encoder, numeric_cols)],
        remainder='passthrough')

    # getting list of column names to map
    columns_to_map = numeric_cols + one_hot_col

    print('columns after preprocessing :', columns_to_map,  '\n')

    # applying encoding on columns in df and creating pipeline
    X_train_enc = pd.DataFrame({col: vals for vals, col in zip(
        preprocessor.fit_transform(X_train_oversampled, y_train).T, columns_to_map)})
    pipe = Pipeline(steps=[("preprocessor", preprocessor)])
    pipe = pipe.fit(X_train_enc, y_train)

    # encoding nucleotides
    for i in range(7):
        X_train_enc['position_' + str(i) + '_A'] = 0
        X_train_enc['position_' + str(i) + '_C'] = 0
        X_train_enc['position_' + str(i) + '_G'] = 0
        X_train_enc['position_' + str(i) + '_T'] = 0
        temp = X_train_enc['nucleotides'].apply(lambda x: x[i])
        X_train_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
        X_train_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
        X_train_enc['position_' + str(i) + '_G'][temp == 'G'] = 1
        X_train_enc['position_' + str(i) + '_T'][temp == 'T'] = 1

    # dropping nucleotides column
    X_train_enc = X_train_enc.drop(columns=['nucleotides'])

    return X_train_enc, pipe

In [36]:
X_train_enc, pipe = encoding_train(X_train_oversampled, y_train)

columns after preprocessing : ['reads_count', 'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean', 'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean', 'nucleotides'] 



C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2671075721.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2671075721.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2671075721.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [37]:
def encoding_test_val(X_test, X_val, test=True):

    one_hot_col = ['nucleotides']

    numeric_cols = ['reads_count','dwelling_time_-1_25',
                    'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                    'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25',
                    'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25',
                    'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                    'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25',
                    'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25',
                    'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                    'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25',
                    'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']   
    columns_to_map = numeric_cols + one_hot_col

    # applying encoding on columns in df
    if test == True:  # for test
        X_test_enc = pd.DataFrame({col: vals for vals, col in zip(
            pipe.transform(X_test).T, columns_to_map)})
        # encoding nucleotides
        for i in range(7):
            X_test_enc['position_' + str(i) + '_A'] = 0
            X_test_enc['position_' + str(i) + '_C'] = 0
            X_test_enc['position_' + str(i) + '_G'] = 0
            X_test_enc['position_' + str(i) + '_T'] = 0
            temp = X_test_enc['nucleotides'].apply(lambda x: x[i])
            X_test_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
            X_test_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
            X_test_enc['position_' + str(i) + '_G'][temp == 'G'] = 1
            X_test_enc['position_' + str(i) + '_T'][temp == 'T'] = 1

        # dropping nucleotides column
        X_test_enc = X_test_enc.drop(columns=['nucleotides'])

        return X_test_enc
    else:  # for validation
        X_val_enc = pd.DataFrame({col: vals for vals, col in zip(
            pipe.transform(X_val).T, columns_to_map)})
        # encoding nucleotides
        for i in range(7):
            X_val_enc['position_' + str(i) + '_A'] = 0
            X_val_enc['position_' + str(i) + '_C'] = 0
            X_val_enc['position_' + str(i) + '_G'] = 0
            X_val_enc['position_' + str(i) + '_T'] = 0
            temp = X_val_enc['nucleotides'].apply(lambda x: x[i])
            X_val_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
            X_val_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
            X_val_enc['position_' + str(i) + '_G'][temp == 'G'] = 1
            X_val_enc['position_' + str(i) + '_T'][temp == 'T'] = 1

        # dropping nucleotides column
        X_val_enc = X_val_enc.drop(columns=['nucleotides'])

        return X_val_enc

In [38]:
X_test_enc = encoding_test_val(X_test, X_val)

C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2122192847.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2122192847.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2122192847.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [39]:
X_test_enc.head()

,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean,position_0_A,position_0_C,position_0_G,position_0_T,position_1_A,position_1_C,position_1_G,position_1_T,position_2_A,position_2_C,position_2_G,position_2_T,position_3_A,position_3_C,position_3_G,position_3_T,position_4_A,position_4_C,position_4_G,position_4_T,position_5_A,position_5_C,position_5_G,position_5_T,position_6_A,position_6_C,position_6_G,position_6_T
0,25.0,0.00609,0.00929,0.011,0.009024,4.61,5.09,5.99,5.2708,114.0,116.0,117.0,115.56,0.00631,0.00996,0.0136,0.011145,4.48,6.73,7.54,6.4036,116.0,118.0,123.0,119.36,0.00498,0.0073,0.0123,0.008693,1.55,2.02,2.58,2.0488,86.1,87.4,89.6,87.736,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
1,24.0,0.006243,0.008255,0.0133,0.009845,4.7925,5.51,6.775,5.563333,113.75,114.5,117.0,115.041667,0.003672,0.00697,0.012,0.008383,4.72,5.58,6.6675,5.600417,117.75,120.5,123.25,120.041667,0.00398,0.005645,0.00785,0.006059,1.8425,2.56,2.9625,2.42825,86.625,88.5,89.925,88.3,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
2,27.0,0.003965,0.00631,0.0081,0.007388,5.65,7.33,8.21,7.343704,120.0,123.0,125.5,122.37037,0.00495,0.00664,0.00964,0.007846,2.87,3.26,3.705,3.423333,93.4,95.4,98.35,95.833333,0.00426,0.00598,0.00727,0.006487,2.215,2.48,2.91,2.601852,86.35,87.3,89.35,87.148148,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
3,32.0,0.005603,0.0084,0.01285,0.009724,4.7275,6.48,8.2875,6.747813,120.0,124.0,126.0,123.21875,0.00493,0.00597,0.00832,0.008302,1.96,2.37,2.8225,2.691875,101.0,102.0,103.25,102.071875,0.003813,0.00469,0.008052,0.006234,1.58,1.935,2.07,1.844531,93.85,95.6,96.975,95.05,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
4,33.0,0.00498,0.00664,0.011,0.009066,2.86,3.04,3.51,3.15,109.0,110.0,111.0,110.112121,0.00532,0.00756,0.00963,0.008135,5.13,6.48,8.05,6.707273,123.0,125.0,126.0,124.515152,0.00398,0.00597,0.00941,0.008685,1.84,2.59,3.25,2.92,88.9,89.9,91.6,90.087879,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0


In [40]:
X_val_enc = encoding_test_val(X_test, X_val, test = False)

C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2122192847.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2122192847.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_27948\2122192847.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [41]:
X_train_25 = X_train_enc.drop(columns=['dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean',
                                        'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_50', 'mean_0_75', 'mean_0_mean',
                                        'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean'])

X_train_50 = X_train_enc.drop(columns=['dwelling_time_-1_25', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'std_-1_25', 'std_-1_75', 'std_-1_mean', 'mean_-1_25', 'mean_-1_75', 'mean_-1_mean',
                                        'dwelling_time_0_25', 'dwelling_time_0_75', 'dwelling_time_0_mean', 'std_0_25', 'std_0_75', 'std_0_mean', 'mean_0_25', 'mean_0_75', 'mean_0_mean',
                                        'dwelling_time_+1_25', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'std_+1_25', 'std_+1_75', 'std_+1_mean', 'mean_+1_25', 'mean_+1_75', 'mean_+1_mean'])

X_train_75 = X_train_enc.drop(columns=['dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_mean', 'std_-1_25', 'std_-1_50', 'std_-1_mean', 'mean_-1_25', 'mean_-1_50', 'mean_-1_mean',
                                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_mean', 'std_0_25', 'std_0_50', 'std_0_mean', 'mean_0_25', 'mean_0_50', 'mean_0_mean',
                                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_mean', 'std_+1_25', 'std_+1_50', 'std_+1_mean', 'mean_+1_25', 'mean_+1_50', 'mean_+1_mean'])

X_train_mean = X_train_enc.drop(columns=['dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'std_-1_25', 'std_-1_50', 'std_-1_75', 'mean_-1_25', 'mean_-1_50', 'mean_-1_75',
                                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'std_0_25', 'std_0_50', 'std_0_75', 'mean_0_25', 'mean_0_50', 'mean_0_75',
                                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'std_+1_25', 'std_+1_50', 'std_+1_75', 'mean_+1_25', 'mean_+1_50', 'mean_+1_75'])

In [47]:
%cd ../../data/new_preprocessed_data

c:\Users\Zoe\DSA4262-ACMXZ\data\new_preprocessed_data


In [48]:
%pwd

'c:\\Users\\Zoe\\DSA4262-ACMXZ\\data\\new_preprocessed_data'

In [49]:
#into archive
percentiles_df.to_parquet("archive/df_all_features.parquet") # df containing all features, before splitting/encoding
df_test.to_parquet("archive/df_test.parquet") # df test containing all features and target, after splitting, before encoding
df_train.to_parquet("archive/df_train.parquet") # df trin containing all features and target, after splitting, before encoding
df_val_id.to_parquet("archive/df_val_id.parquet") # df val containing all features, identity cols and target, after splitting, before encoding
df_val.to_parquet("archive/df_val.parquet") # df val containing all features and target, after splitting, before encoding
X_test.to_parquet("archive/X_test.parquet") # df test containing all features, after splitting, before encoding
X_train.to_parquet("archive/X_train.parquet") # df train containing all features, after splitting, before encoding
X_val.to_parquet("archive/X_val.parquet") # df val containing all features, after splitting, before encoding

#into test
X_test_enc.to_parquet("test/X_test_enc.parquet") # df test containing all features, after encoding and oversampling
y_test.to_parquet("test/y_test.parquet") # target test, after splitting, before encoding

#into training
X_train_25.to_parquet("training/X_train_25.parquet")  # final df containing 25 percentile features for modelling
X_train_50.to_parquet("training/X_train_50.parquet") # final df containing 50 percentile features for modelling
X_train_75.to_parquet("training/X_train_75.parquet") # final df containing 75 percentile features for modelling
X_train_enc.to_parquet("training/X_train_enc.parquet") # df train containing all features, after encoding and oversampling
X_train_mean.to_parquet("training/X_train_mean.parquet") # final df containing mean features for modelling
y_train.to_parquet("training/y_train.parquet") # target train, after splitting, before encoding

#into validation
X_val_enc.to_parquet("validation/X_val_enc.parquet") # df val containing all features, after encoding and oversampling
y_val.to_parquet("validation/y_val.parquet") # target val, after splitting, before encoding